In [54]:
# Loading Libraries
import pandas as pd
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import log_hyperu as hyperu
import tgr as tgr

In [22]:
# Generating the data sets

torch.manual_seed(12345)

# Structure of the data sets
## Set X_A: Sparse Coefficients with many data points
## Set X_B: Dense Coefficients with many data points
## Set X_C: Sparse Coefficients with few data points
## Set X_D: Dense Coefficients with few data points


### Set X_A
variables = 4
sample = 100
true_coefs = torch.tensor([[0.5],[0.2],[5.3],[4.2]])
X = torch.randn(sample, variables)
Y = X @ true_coefs + torch.randn(sample, 1) * 0.1 

In [55]:
trained_model2, coefs, loss_of_optimization = tgr.TripleGammaModel(X, Y, 0.05, 0.6, 0.1, 2, True) # Covariates, Targets, Penalty, a, c, kappa, normalization=True
coefficients = trained_model2.linear.weight.detach().numpy()
intercept = trained_model2.linear.bias.item()

print("Coefficients General Function:", coefficients)
print("Intercept General Function:", intercept)

Epoch [100/1000], Loss: 28.541316986083984
Epoch [200/1000], Loss: 19.670513153076172
Epoch [300/1000], Loss: 12.691256523132324
Epoch [400/1000], Loss: 7.757358551025391
Epoch [500/1000], Loss: 4.368696689605713
Epoch [600/1000], Loss: 2.1880898475646973
Epoch [700/1000], Loss: 1.1026486158370972
Epoch [800/1000], Loss: 0.8473856449127197
Epoch [900/1000], Loss: 0.8290976881980896
Epoch [1000/1000], Loss: 0.808399498462677
Coefficients General Function: [[ 8.1124157e-04 -6.7135412e-04  5.1900401e+00  4.1617389e+00]]
Intercept General Function: 0.011507042683660984
